In [1]:
import pandas as pd
import numpy as np
import tarfile
import io

tar = tarfile.open('../data/raw/20181120_track_features.tar.gz', 'r:gz')
csv_files = tar.getnames()

df_list = []

for csv_file in [csv_files[2], csv_files[4]]:
    csv_contents = tar.extractfile(csv_file).read()
    df_list.append(pd.read_csv(io.BytesIO(csv_contents), encoding='utf8'))

tf_df = pd.concat(df_list, ignore_index=True)
del df_list

In [2]:
tf_df.head()

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,t_2e8f4b71-8a0b-4b9c-b7d8-fb5208e87f9f,326.013336,1971,99.582885,0.716209,0.366495,0.332605,0.439835,5.805774,0.238847,...,4,0.223395,0.146012,-0.706908,0.259496,0.481157,0.238427,-0.098389,-0.254960,-0.227383
1,t_dae2ec0e-ec7b-4b3e-b60c-4a884d0eccb0,147.813324,1963,97.272035,0.839460,0.362212,0.389829,0.507580,6.845427,0.420476,...,4,0.484702,0.039554,-0.539554,0.105141,0.692589,0.226047,-0.468162,0.164389,-0.769024
2,t_cf0164dd-1531-4399-bfa6-dec19cd1fedc,110.400002,1974,99.620384,0.054673,0.495002,0.589378,0.552311,9.361949,0.842938,...,4,0.818441,0.083863,-0.242108,-0.014258,0.096396,0.417641,-0.050576,-0.204757,-0.172563
3,t_0f90acc7-d5c5-4e53-901d-55610fbd090c,237.653336,1988,96.796830,0.042606,0.389634,0.359044,0.585673,6.068578,0.665398,...,4,0.594829,0.192498,0.340039,0.034846,-0.389794,0.518381,0.185008,-0.079907,-0.016978
4,t_36b9ad02-095a-443d-a697-6c7285d9410a,174.600006,1987,97.905891,0.249982,0.513640,0.485435,0.635095,7.198735,0.408715,...,4,0.591289,0.270586,-0.411061,0.165898,0.225652,0.335518,-0.036643,-0.016300,-0.446870


In [3]:
tf_df = tf_df.merge(pd.read_csv('../data/interim/all_data/mbKMeans100clusters.csv', usecols=['track_id','clus']))

In [4]:
log_df = pd.read_csv('../data/raw/training_set/log_0_20180715_000000000000.csv')
log_df = log_df.loc[log_df['session_position']<(log_df['session_length']/2)] # only use the 1st half
log_df = pd.get_dummies(log_df, columns=['hist_user_behavior_reason_end', 'hist_user_behavior_reason_start', 'context_type'])

tf_df = pd.get_dummies(tf_df, columns=['key','time_signature','mode'])

df = log_df.merge(tf_df, left_on = 'track_id_clean', right_on = 'track_id')
df.sort_values(by = ['session_id', 'session_position'],inplace = True)

bool_col = ['skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch',
       'no_pause_before_play', 'short_pause_before_play',
       'long_pause_before_play', 'hist_user_behavior_is_shuffle','premium',
       'hist_user_behavior_reason_end_backbtn',
       'hist_user_behavior_reason_end_clickrow',
       'hist_user_behavior_reason_end_endplay',
       'hist_user_behavior_reason_end_fwdbtn',
       'hist_user_behavior_reason_end_logout',
       'hist_user_behavior_reason_end_remote',
       'hist_user_behavior_reason_end_trackdone',
       'hist_user_behavior_reason_start_appload',
       'hist_user_behavior_reason_start_backbtn',
       'hist_user_behavior_reason_start_clickrow',
       'hist_user_behavior_reason_start_endplay',
       'hist_user_behavior_reason_start_fwdbtn',
       'hist_user_behavior_reason_start_playbtn',
       'hist_user_behavior_reason_start_remote',
       'hist_user_behavior_reason_start_trackdone',
       'hist_user_behavior_reason_start_trackerror', 'context_type_catalog',
       'context_type_charts', 'context_type_editorial_playlist',
       'context_type_personalized_playlist', 'context_type_radio',
       'context_type_user_collection', 
       'key_0', 'key_1', 'key_2', 'key_3', 'key_4',
       'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11',
       'time_signature_0', 'time_signature_1', 'time_signature_3',
       'time_signature_4', 'time_signature_5', 'mode_major', 'mode_minor']

df[bool_col] = df[bool_col].astype('bool')
df.head()
del log_df, tf_df

In [5]:
# from functions import featureAugment

In [5]:
import warnings
warnings.simplefilter("ignore")

from dask.dataframe import from_pandas
ddf = from_pandas(df, npartitions=3)



In [6]:
import dask.dataframe as dd
median_fun = dd.Aggregation(
    name='median_fun',
    chunk=lambda s: s.median(), # this computes the median on each partition
    agg=lambda s0: s0.sum() # this combines results across partitions; the input should just be a list of length 1
)  

iqr_fun = dd.Aggregation(
    name='iqr_fun',
    chunk=lambda s: s.quantile(.75)-s.quantile(.25), # this computes the median on each partition
    agg=lambda s0: s0.sum() # this combines results across partitions; the input should just be a list of length 1
)  

skew_fun = dd.Aggregation(
    name='skew_fun',
    chunk=lambda s: s.skew(), # this computes the median on each partition
    agg=lambda s0: s0.sum() # this combines results across partitions; the input should just be a list of length 1
)

range_fun = dd.Aggregation(
    name='range_fun',
    chunk=lambda s: s.max()-s.min(), # this computes the median on each partition
    agg=lambda s0: s0.sum() # this combines results across partitions; the input should just be a list of length 1
)

In [8]:
col_FA = ['session_position', 'session_length', 
       'skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch',
       'no_pause_before_play', 'short_pause_before_play',
       'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
       'hist_user_behavior_n_seekback', 'hist_user_behavior_is_shuffle',
       'hour_of_day', 'premium',
       'hist_user_behavior_reason_end_backbtn',
       'hist_user_behavior_reason_end_clickrow',
       'hist_user_behavior_reason_end_endplay',
       'hist_user_behavior_reason_end_fwdbtn',
       'hist_user_behavior_reason_end_logout',
       'hist_user_behavior_reason_end_remote',
       'hist_user_behavior_reason_end_trackdone',
       'hist_user_behavior_reason_start_appload',
       'hist_user_behavior_reason_start_backbtn',
       'hist_user_behavior_reason_start_clickrow',
       'hist_user_behavior_reason_start_endplay',
       'hist_user_behavior_reason_start_fwdbtn',
       'hist_user_behavior_reason_start_playbtn',
       'hist_user_behavior_reason_start_remote',
       'hist_user_behavior_reason_start_trackdone',
       'hist_user_behavior_reason_start_trackerror', 'context_type_catalog',
       'context_type_charts', 'context_type_editorial_playlist',
       'context_type_personalized_playlist', 'context_type_radio',
       'context_type_user_collection', 'duration', 'release_year',
       'us_popularity_estimate', 'acousticness', 'beat_strength', 'bounciness',
       'danceability', 'dyn_range_mean', 'energy', 'flatness',
       'instrumentalness', 'liveness', 'loudness', 'mechanism', 'organism',
       'speechiness', 'tempo', 'valence', 'acoustic_vector_0',
       'acoustic_vector_1', 'acoustic_vector_2', 'acoustic_vector_3',
       'acoustic_vector_4', 'acoustic_vector_5', 'acoustic_vector_6',
       'acoustic_vector_7', 'key_0', 'key_1', 'key_2', 'key_3', 'key_4',
       'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11',
       'time_signature_0', 'time_signature_1', 'time_signature_3',
       'time_signature_4', 'time_signature_5', 'mode_major', 'mode_minor']

In [14]:
from timeit import default_timer as timer #to see how long the computation will take
start = timer()

aa = ddf.groupby(['session_id'])[col_FA].agg(['mean'])

print('Runtime: %0.2fs' % (timer() - start))

Runtime: 89.09s


In [15]:
aa

,session_position,session_length,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,premium,hist_user_behavior_reason_end_backbtn,hist_user_behavior_reason_end_clickrow,hist_user_behavior_reason_end_endplay,hist_user_behavior_reason_end_fwdbtn,hist_user_behavior_reason_end_logout,hist_user_behavior_reason_end_remote,hist_user_behavior_reason_end_trackdone,hist_user_behavior_reason_start_appload,hist_user_behavior_reason_start_backbtn,hist_user_behavior_reason_start_clickrow,hist_user_behavior_reason_start_endplay,hist_user_behavior_reason_start_fwdbtn,hist_user_behavior_reason_start_playbtn,hist_user_behavior_reason_start_remote,hist_user_behavior_reason_start_trackdone,hist_user_behavior_reason_start_trackerror,context_type_catalog,context_type_charts,context_type_editorial_playlist,context_type_personalized_playlist,context_type_radio,context_type_user_collection,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,liveness,loudness,mechanism,organism,speechiness,tempo,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,time_signature_0,time_signature_1,time_signature_3,time_signature_4,time_signature_5,mode_major,mode_minor
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
def featureAugment(df_history, cols):
    import pandas as pd
    import numpy as np
    df_fa = pd.DataFrame()
    
    # split the data based on skip_2
    df_history_T = df_history.loc[df_history['skip_2']==True]
    df_history_F = df_history.loc[df_history['skip_2']==False]
        
    for c in cols:
        df_fa[c+'_mean'] = df_history.groupby(['session_id'])[c].mean()
        df_fa[c+'_std'] = df_history.groupby(['session_id'])[c].std()
        df_fa[c+'_skew'] = df_history.groupby(['session_id'])[c].skew()
        df_fa[c+'_median'] = df_history.groupby(['session_id'])[c].median()
        
        df_fa[c+'_T_mean'] = df_history_T.groupby(['session_id'])[c].mean()
        df_fa[c+'_T_std'] = df_history_T.groupby(['session_id'])[c].std()
        df_fa[c+'_T_skew'] = df_history_T.groupby(['session_id'])[c].skew()
        df_fa[c+'_T_median'] = df_history_T.groupby(['session_id'])[c].median()
        
        df_fa[c+'_F_mean'] = df_history_F.groupby(['session_id'])[c].mean()
        df_fa[c+'_F_std'] = df_history_F.groupby(['session_id'])[c].std()
        df_fa[c+'_F_skew'] = df_history_F.groupby(['session_id'])[c].skew()
        df_fa[c+'_F_median'] = df_history_F.groupby(['session_id'])[c].median()
        
        # correlations
        df_fa[c+'_PearsonTrend'] = df_history.groupby(['session_id'])[c, 'session_position'].corr().iloc[0::2,-1].reset_index(level=1, drop=True)
        df_fa[c+'_SpearTrend'] = df_history.groupby(['session_id'])[c, 'session_position'].corr('spearman').iloc[0::2,-1].reset_index(level=1, drop=True)
        df_fa[c+'_KendallTrend'] = df_history.groupby(['session_id'])[c, 'session_position'].corr('kendall').iloc[0::2,-1].reset_index(level=1, drop=True)

        df_fa[c+'_T_PearsonTrend'] = df_history_T.groupby(['session_id'])[c, 'session_position'].corr().iloc[0::2,-1].reset_index(level=1, drop=True)
        df_fa[c+'_T_SpearTrend'] = df_history_T.groupby(['session_id'])[c, 'session_position'].corr('spearman').iloc[0::2,-1].reset_index(level=1, drop=True)
        df_fa[c+'_T_KendallTrend'] = df_history_T.groupby(['session_id'])[c, 'session_position'].corr('kendall').iloc[0::2,-1].reset_index(level=1, drop=True)

        df_fa[c+'_F_PearsonTrend'] = df_history_F.groupby(['session_id'])[c, 'session_position'].corr().iloc[0::2,-1].reset_index(level=1, drop=True)
        df_fa[c+'_F_SpearTrend'] = df_history_F.groupby(['session_id'])[c, 'session_position'].corr('spearman').iloc[0::2,-1].reset_index(level=1, drop=True)
        df_fa[c+'_F_KendallTrend'] = df_history_F.groupby(['session_id'])[c, 'session_position'].corr('kendall').iloc[0::2,-1].reset_index(level=1, drop=True)
        
        if df_history[c].dtypes.name != 'bool':
            df_fa[c+'_max'] = df_history.groupby(['session_id'])[c].max()
            df_fa[c+'_min'] = df_history.groupby(['session_id'])[c].min()
            df_fa[c+'_range'] = np.subtract(df_fa[c+'_max'], df_fa[c+'_min'], dtype=np.float32)
            df_fa[c+'_q25'] = df_history.groupby(['session_id'])[c].quantile(.25)
            df_fa[c+'_q75'] = df_history.groupby(['session_id'])[c].quantile(.75)
            df_fa[c+'_iqr'] = np.subtract(df_fa[c+'_q75'], df_fa[c+'_q25'], dtype=np.float32)
            
            df_fa[c+'_T_max'] = df_history_T.groupby(['session_id'])[c].max()
            df_fa[c+'_T_min'] = df_history_T.groupby(['session_id'])[c].min()
            df_fa[c+'_T_range'] = np.subtract(df_fa[c+'_T_max'], df_fa[c+'_T_min'], dtype=np.float32)
            df_fa[c+'_T_q25'] = df_history_T.groupby(['session_id'])[c].quantile(.25)
            df_fa[c+'_T_q75'] = df_history_T.groupby(['session_id'])[c].quantile(.75)
            df_fa[c+'_T_iqr'] = np.subtract(df_fa[c+'_T_q75'], df_fa[c+'_T_q25'], dtype=np.float32)
            
            df_fa[c+'_F_max'] = df_history_F.groupby(['session_id'])[c].max()
            df_fa[c+'_F_min'] = df_history_F.groupby(['session_id'])[c].min()
            df_fa[c+'_F_range'] = np.subtract(df_fa[c+'_F_max'], df_fa[c+'_F_min'], dtype=np.float32)
            df_fa[c+'_F_q25'] = df_history_F.groupby(['session_id'])[c].quantile(.25)
            df_fa[c+'_F_q75'] = df_history_F.groupby(['session_id'])[c].quantile(.75)
            df_fa[c+'_F_iqr'] = np.subtract(df_fa[c+'_F_q75'], df_fa[c+'_F_q25'], dtype=np.float32)
            
#             df_fa[c+'_mode'] = df_history.groupby(['session_id'])[c].agg(pd.Series.mode)

    return df_fa

In [7]:
from timeit import default_timer as timer #to see how long the computation will take
start = timer()



df_fa = featureAugment(df, col_FA)
print('Runtime: %0.2fs' % (timer() - start))

/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:207: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fa[c+'_PearsonTrend'] = df_history.groupby(['session_id'])[c, 'session_position'].corr().iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:208: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fa[c+'_SpearTrend'] = df_history.groupby(['session_id'])[c, 'session_position'].corr('spearman').iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:209: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fa[c+'_KendallTrend'] = df_history.groupby(['session_id'])[c, 'sessi

/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:213: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fa[c+'_T_KendallTrend'] = df_history_T.groupby(['session_id'])[c, 'session_position'].corr('kendall').iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:215: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fa[c+'_F_PearsonTrend'] = df_history_F.groupby(['session_id'])[c, 'session_position'].corr().iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:216: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fa[c+'_F_SpearTrend'] = df_history_F.groupby(['session_id']

/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:209: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fa[c+'_KendallTrend'] = df_history.groupby(['session_id'])[c, 'session_position'].corr('kendall').iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:211: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fa[c+'_T_PearsonTrend'] = df_history_T.groupby(['session_id'])[c, 'session_position'].corr().iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:211: PerformanceWarning: DataFrame is highly f

/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:198: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fa[c+'_T_skew'] = df_history_T.groupby(['session_id'])[c].skew()
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:199: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fa[c+'_T_median'] = df_history_T.groupby(['session_id'])[c].median()
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:201: PerformanceWarning: DataFrame is highl

/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:215: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fa[c+'_F_PearsonTrend'] = df_history_F.groupby(['session_id'])[c, 'session_position'].corr().iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:216: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fa[c+'_F_SpearTrend'] = df_history_F.groupby(['session_id'])[c, 'session_position'].corr('spearman').iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:216: PerformanceWarning: DataFrame is highl

...

/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fa[c+'_SpearTrend'] = df_history.groupby(['session_id'])[c, 'session_position'].corr('spearman').iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:209: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fa[c+'_KendallTrend'] = df_history.groupby(['session_id'])[c, 'session_position'].corr('kendall').iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:209: PerformanceWarning: DataFrame is high

/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:197: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fa[c+'_T_std'] = df_history_T.groupby(['session_id'])[c].std()
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:198: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fa[c+'_T_skew'] = df_history_T.groupby(['session_id'])[c].skew()
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:199: PerformanceWarning: DataFrame is highly frag

/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:215: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fa[c+'_F_PearsonTrend'] = df_history_F.groupby(['session_id'])[c, 'session_position'].corr().iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:216: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fa[c+'_F_SpearTrend'] = df_history_F.groupby(['session_id'])[c, 'session_position'].corr('spearman').iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:216: PerformanceWarning: DataFrame is highl

/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fa[c+'_SpearTrend'] = df_history.groupby(['session_id'])[c, 'session_position'].corr('spearman').iloc[0::2,-1].reset_index(level=1, drop=True)
/Volumes/AC8888/Capstone_Spotify-Sequential-Skip-Prediction/notebooks/functions.py:209: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fa[c+'_KendallTrend'] = df_history.groupby(['session_id'])[c, 'session_position'].corr('kendall').iloc[0::2,-1].reset_index(level=1, drop=True)


KeyboardInterrupt: 